#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링 
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.

import tensorflow as tf
from tensorflow import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - data01_test.csv 를 불러와 'new_data' 이름으로 저장합니다.


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/AIVLE/0412-0414/data01_train.csv')
data = data.drop('subject', axis=1)

In [ ]:
new_data = pd.read_csv('/content/drive/MyDrive/AIVLE/0412-0414/data01_train.csv')

#일단은 임시로 train다시대입

## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

 **세부 요구사항**
    - Target인 Activity를 is_dynamic 변수로 변경하시오.
        * 값 0 : STANDING, SITTING, LAYING
        * 값 1 : WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS
    - is_dynamic을 예측하기 위한 기본 모델을 생성하시오.
        * x, is_dynamic을 이용하여 train과 validation으로 데이터 분할
        * 기본 모델링
    - is_dynamic 을 구분하는데 중요한 변수 상위 5를 분석하시오.
    - is_dynamic 을 구분하는데 중요한 변수 그룹을 분석하시오.
        * sensor별
        * sensor+agg 별

In [ ]:
target = 'Activity'

In [ ]:
data[target]

0                 STANDING
1                   LAYING
2                 STANDING
3                  WALKING
4       WALKING_DOWNSTAIRS
               ...        
5876               SITTING
5877      WALKING_UPSTAIRS
5878                LAYING
5879      WALKING_UPSTAIRS
5880               SITTING
Name: Activity, Length: 5881, dtype: object

In [ ]:
data['Activity_dynamic'] = data[target]

In [ ]:
data['Activity_dynamic']

0                 STANDING
1                   LAYING
2                 STANDING
3                  WALKING
4       WALKING_DOWNSTAIRS
               ...        
5876               SITTING
5877      WALKING_UPSTAIRS
5878                LAYING
5879      WALKING_UPSTAIRS
5880               SITTING
Name: Activity_dynamic, Length: 5881, dtype: object

In [ ]:
data['Activity_dynamic'] = data['Activity_dynamic'].replace('STANDING', 0)
data['Activity_dynamic'] = data['Activity_dynamic'].replace('SITTING', 0)
data['Activity_dynamic'] = data['Activity_dynamic'].replace('LAYING', 0)
data['Activity_dynamic'] = data['Activity_dynamic'].replace('WALKING', 1)
data['Activity_dynamic'] = data['Activity_dynamic'].replace('WALKING_UPSTAIRS', 1)
data['Activity_dynamic'] = data['Activity_dynamic'].replace('WALKING_DOWNSTAIRS', 1)

In [ ]:
data['Activity_dynamic'].value_counts(), data['Activity'].value_counts()

(0    3234
 1    2647
 Name: Activity_dynamic, dtype: int64,
 LAYING                1115
 STANDING              1087
 SITTING               1032
 WALKING                998
 WALKING_UPSTAIRS       858
 WALKING_DOWNSTAIRS     791
 Name: Activity, dtype: int64)

In [ ]:
X = data.drop([target, 'Activity_dynamic'], axis=1)

In [ ]:
Y1 = data[target]
Y2 = data['Activity_dynamic']

In [ ]:
X.shape, Y1.shape, Y2.shape

((5881, 561), (5881,), (5881,))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_X, val_X, train_Y1, val_Y1, train_Y2, val_Y2 =\
    train_test_split(X, Y1, Y2, stratify=Y1, random_state=42, test_size=0.2)

In [ ]:
train_X.shape, train_Y1.shape, train_Y2.shape, val_X.shape, val_Y1.shape, val_Y2.shape

((4704, 561), (4704,), (4704,), (1177, 561), (1177,), (1177,))

## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

#### 1) 알고리즘1 : 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
knn = KNeighborsClassifier()
knn.fit(train_X, train_Y2)
pred_Y = knn.predict(val_X)
knn.score(val_X, val_Y2)

0.9991503823279524

#### 2) 알고리즘2 : 

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
class_n2 = len(set(train_Y2))

train_Y2 = to_categorical(train_Y2, class_n2)
val_Y2 = to_categorical(val_Y2, class_n2)

In [ ]:
train_Y2.shape, val_Y2.shape

((4704, 2), (1177, 2))

In [ ]:
train_X.shape

(4704, 561)

In [ ]:
# 세션 클리어
keras.backend.clear_session()

# 레이어 엮기
il = keras.layers.Input(shape=(561,))

hl = keras.layers.Dense(1024, activation='relu',)(il)
hl = keras.layers.Dense(512, activation='relu',)(il)
hl = keras.layers.Dense(256, activation='relu',)(il)
hl = keras.layers.Dense(256, activation='relu',)(il)
hl = keras.layers.Dense(128, activation='relu',)(il)
hl = keras.layers.Dense(64, activation='relu',)(il)

ol = keras.layers.Dense(2, activation='softmax')(hl)

# 모델 시작 끝 지정
model = keras.models.Model(il, ol)

# 컴파일
model.compile(loss='binary_crossentropy', metrics=['accuracy'],
              optimizer='adam')

# 요약
model.summary()

model.fit

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 561)]             0         
                                                                 
 dense_5 (Dense)             (None, 64)                35968     
                                                                 
 dense_6 (Dense)             (None, 2)                 130       
                                                                 
Total params: 36,098
Trainable params: 36,098
Non-trainable params: 0
_________________________________________________________________


<bound method Model.fit of <keras.engine.functional.Functional object at 0x7f09a0274df0>>

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,                
                   patience=5,                 
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
model.fit(train_X, train_Y2, validation_split=0.2, epochs=1000, callbacks=[es])

Epoch 1/1000
118/118 [==============================] - 1s 4ms/step - loss: 1.7079e-08 - accuracy: 1.0000 - val_loss: 1.5595e-08 - val_accuracy: 1.0000
Epoch 2/1000
118/118 [==============================] - 0s 3ms/step - loss: 1.5569e-08 - accuracy: 1.0000 - val_loss: 1.2749e-08 - val_accuracy: 1.0000
Epoch 3/1000
118/118 [==============================] - 0s 3ms/step - loss: 1.4631e-08 - accuracy: 1.0000 - val_loss: 1.1929e-08 - val_accuracy: 1.0000
Epoch 4/1000
118/118 [==============================] - 0s 3ms/step - loss: 1.4275e-08 - accuracy: 1.0000 - val_loss: 1.2291e-08 - val_accuracy: 1.0000
Epoch 5/1000
118/118 [==============================] - 0s 3ms/step - loss: 1.3258e-08 - accuracy: 1.0000 - val_loss: 1.1883e-08 - val_accuracy: 1.0000
Epoch 6/1000
118/118 [==============================] - 0s 3ms/step - loss: 1.2333e-08 - accuracy: 1.0000 - val_loss: 1.0722e-08 - val_accuracy: 1.0000
Epoch 7/1000
118/118 [==============================] - 0s 3ms/step - loss: 1.2009e-08 -

### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
stop = ['LAYING', 'SITTING', 'STANDING']

In [ ]:
data_stop = data[data[target].isin(stop)]

In [ ]:
data_stop[target].value_counts()

LAYING      1115
STANDING    1087
SITTING     1032
Name: Activity, dtype: int64

In [ ]:
data_stop[target] = data_stop[target].replace('STANDING', 2)
data_stop[target] = data_stop[target].replace('SITTING', 1)
data_stop[target] = data_stop[target].replace('LAYING', 0)

In [ ]:
sX = data_stop.drop(target, axis=1)
sY = data_stop[target]

In [ ]:
data_stop[target]

In [ ]:
train_sX, val_sX, train_sY, val_sY=\
    train_test_split(sX, sY, stratify=sY, random_state=42, test_size=0.2)

In [ ]:
sclass_n = len(set(train_sY))

train_sY = to_categorical(train_sY, sclass_n)
val_sY = to_categorical(val_sY, sclass_n)

In [ ]:
train_sX.shape

(2587, 562)

In [ ]:
# 세션 클리어
keras.backend.clear_session()

# 레이어 엮기
il = keras.layers.Input(shape=(562,))

hl = keras.layers.Dense(256, activation='relu',)(il)
hl = keras.layers.Dense(256, activation='relu',)(il)
hl = keras.layers.Dense(128, activation='relu',)(il)
hl = keras.layers.Dense(64, activation='relu',)(il)

ol = keras.layers.Dense(3, activation='softmax')(hl)

# 모델 시작 끝 지정
model_s = keras.models.Model(il, ol)

# 컴파일
model_s.compile(loss='categorical_crossentropy', metrics=['accuracy'],
              optimizer='adam')

# 요약
model_s.summary()

model_s.fit(train_sX, train_sY, validation_split=0.2, epochs=1000, callbacks=[es])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 562)]             0         
                                                                 
 dense_3 (Dense)             (None, 64)                36032     
                                                                 
 dense_4 (Dense)             (None, 3)                 195       
                                                                 
Total params: 36,227
Trainable params: 36,227
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
65/65 [==============================] - 1s 6ms/step - loss: 0.6655 - accuracy: 0.7487 - val_loss: 0.4400 - val_accuracy: 0.8784
Epoch 2/1000
65/65 [==============================] - 0s 3ms/step - loss: 0.3441 - accuracy: 0.8821 - val_loss: 0.3332 - val_accuracy: 0.8649
Epoch 3/1000
65/65 [=================

### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
move = ['WALKING', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']

In [ ]:
data_move = data[data[target].isin(move)]

In [ ]:
data_move[target].value_counts()

WALKING               998
WALKING_UPSTAIRS      858
WALKING_DOWNSTAIRS    791
Name: Activity, dtype: int64

In [ ]:
data_move[target] = data_move[target].replace('WALKING', 0)
data_move[target] = data_move[target].replace('WALKING_DOWNSTAIRS', 1)
data_move[target] = data_move[target].replace('WALKING_UPSTAIRS', 2)

In [ ]:
mX = data_move.drop(target, axis=1)
mY = data_move[target]

In [ ]:
data_move[target]

In [ ]:
train_mX, val_mX, train_mY, val_mY =\
  train_test_split(mX, mY, stratify = mY, random_state = 42, test_size= 0.2)

In [ ]:
mclass_n = len(set(train_mY))

train_mY = to_categorical(train_mY, mclass_n)
val_mY = to_categorical(val_mY, mclass_n)

In [ ]:
val_mY

In [ ]:
# 세션 클리어
keras.backend.clear_session()

# 레이어 엮기
il = keras.layers.Input(shape=(562,))

hl = keras.layers.Dense(256, activation='relu',)(il)
hl = keras.layers.Dense(256, activation='relu',)(il)
hl = keras.layers.Dense(128, activation='relu',)(il)
hl = keras.layers.Dense(64, activation='relu',)(il)

ol = keras.layers.Dense(3, activation='softmax')(hl)

# 모델 시작 끝 지정
model_m = keras.models.Model(il, ol)

# 컴파일
model_m.compile(loss='categorical_crossentropy', metrics=['accuracy'],
              optimizer='adam')

# 요약
model_m.summary()

model_m.fit(train_mX, train_mY, validation_split=0.2, epochs=1000, callbacks=[es])

* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들기

In [ ]:
model_m.fit(train_mX, train_mY, validation_split=0.2, epochs=1000, callbacks=[es])